In [3]:
from pyspark.sql import SparkSession
app_name = "normalize_example"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [4]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [29]:
df = spark.createDataFrame([['A',1],['A',2],['A',3],['B',2],['B',4]])

In [9]:
df.show()

+---+---+
| _1| _2|
+---+---+
|  A|  1|
|  A|  2|
|  A|  3|
|  B|  2|
|  B|  4|
+---+---+



In [32]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

def z_score(c, w):
    return (F.col(c) - F.mean(c).over(w)) / F.stddev(c).over(w)

def min_max(c, w):
    return (F.col(c) - F.min(c).over(w)) / (F.max(c).over(w) - F.min(c).over(w))


w = Window.partitionBy("_1")

df.select("_1", z_score("_2",w).alias("degree_normalized"), min_max("_2",w).alias("degree_min_max")).show()


'''
categories = ['station','s']
w = Window.partitionBy(*categories)

df.select(z_score("degree",w).alias("degree_normalized"), min_max("degree",w).alias("degree_min_max"),*categories).show()
'''

+---+-------------------+--------------+
| _1|  degree_normalized|degree_min_max|
+---+-------------------+--------------+
|  B|-0.7071067811865475|           0.0|
|  B| 0.7071067811865475|           1.0|
|  A|               -1.0|           0.0|
|  A|                0.0|           0.5|
|  A|                1.0|           1.0|
+---+-------------------+--------------+

